<DIV ALIGN=CENTER>

# Unix Data Exploration
## Professor Robert J. Brunner
  
</DIV>  
-----
-----

## Introduction

One of the easiest ways to explore a data set is often one of the best.
Working at the Unix command line allows you to leverage the many tools
that are built-in to the Unix operating system, many of which were
designed to rapidly process data. In this lesson, we review some of the
basic Unix commands that are useful for exploring data at the command
prompt. This includes file viewing tools as well as data processing
tools.

-----

### Data

Before we can use the Unix tools, we need to first identify a data set.
If you are working in the JupyterHub Server, the [airline flight][af]
data is already pre-cached in the `data` folder. The main directory
includes the flight data for the year 2001 and three supplementary data
sets: the airports data, the carrier codes, and plane data. In the
`data/raw` directory is the flight data for the years 1987-2008.

![Docker sdata](images/docker-sdata.png)

If you are working on your laptop, you will need to want these data
files and store them in the data directory inside your shared folder,
for example, `/home/data_scientist/rppdm/data`. To grab these files, you
can execute the following Unix commands, either in an IPython Notebook
cell (by preceding them with an exclamation character, `!`), by opening
a terminal from your Jupyter server, or by executing them inside a
Docker container.

```console
$ mkdir data
$ cd data
$ wget http://stat-computing.org/dataexpo/2009/2001.csv.bz2
$ bzip2 -d 2001.csv.bz2
$ wget http://stat-computing.org/dataexpo/2009/airports.csv
$ wget http://stat-computing.org/dataexpo/2009/carriers.csv 
$ wget http://stat-computing.org/dataexpo/2009/plane-data.csv
$ ls -la
```

When complete, you should have the following files in your Docker
container:

![Docker data](images/docker-data.png)

-----
[af]: http://stat-computing.org/dataexpo/2009/

### Verifying Data

Once you have data, the next step is to begin exploring it. When working
at the Unix command prompt, you can easily view the beginning and end of
a data file by using the `head` and `tail` commands, respectively. A
useful flag to include with  these commands is `-n` where `n` is the
number of lines you want to display. By default, these commands
generally display ten lines, which may be OK or not, depending on the
number of columns in the data set. When looking at the beginning of a
data set, for example a _CSV_ file, one often wants to look at the first
two rows. This is because these types of files often have a header row
that lists the column names as the first row. Likewise, it is often a
good idea to look at the last few lines of a file to ensure there was no
corruption in the data transport process.

```console
$ head -2 2001.csv
$ head airports.csv
$ tail -2 2001.csv
$ tail -5 carriers.csv
```

![Docker viewing data](images/docker-view.png)

You also can scroll through the contents of a file, which can be useful
to quickly get a sense of the types stored in each column as well as the
general range of each column. The two Unix commands that are most
relevant for this include:

- `cat`: which displays the contents of a file continuously to the screen
- 'less`: which allows a paginated view of a file's contents (see also
the `more` command).

For example, the following screen shows the contents of the 2001.csv
file.

![Docker less view](images/docker-less.png)

In this screenshot, the contents of the first few rows show the presence
of add characters, in this particular case the characters `<E4><E6>`,
which indicate that this data file likely contains characters in a
special encoding. In this case, the encoding, which can be found by
digging around on the source data site, is _Latin-1_, which we will need
to use whenever we read raw data from the `2001.csv` file (or any other
year's flight data).

-----

### Simple Data Processing

In some cases, we simply want to know the number of lines (or rows) in a
data file. The `wc` command can be used to obtain this count. By
default, `wc` displays multiple values. If you simply want the number of
lines you need to use the `-l` flag. For example, to count the number of
lines in the `2001.csv` file:

```console
$ wc -l 2001.csv
5967781 2001.csv
```

The output indicates that there are 5,967,781 rows in this file. Since
the first line was a header row, the file contains information on
5,967,780 flights for the year 2001. 

We can use `wc` in combination with the `grep` command to quickly find
the number of rows that might contain a particular set of characters.
For example, we can find the number of flights in or out of O'hare by
using `grep` to find rows that contain `ORD` (the airport code for
O'Hare), and use a Unix pipe to connect the output of this command into
the input of the `wc command:

```console
$ grep ORD 2001.csv | wc -l
682636
```

Thus, we should expect that O'Hare handled 682, 636 flights in 2001. We
can extend this by connecting multiple `grep` commands together to find
out how many flights were between O'Hare and Willard airport in Savoy.

```console
$ grep ORD 2001.csv | grep CMI | wc -l
3652
```

Of particular interest is how fast these commands operate, which we can
see by executing them in an Ipython Notebook cell.

In [1]:
%time !grep ORD /home/data_scientist/rppdm/data/2001.csv | grep CMI | wc -l

3652
CPU times: user 10 ms, sys: 10 ms, total: 20 ms
Wall time: 3.45 s


Amazingly, these simple Unix commands counted the number of flights
between these two airports in less than 4 seconds (Wall time refers to
what a clock on the wall would report).

-----

### Basic Data Processing

There are a number of Unix commands that can perform basic data
processing, such as sorting rows (`sort`), removing duplicate rows
(`uniq`), cutting out columns or converting character sequences. To
demonstrate, we can use the `cut` command to pull specific columns out
of a file. Several flags are useful with the `cut` command. First is the
`-d` flag, which is used to indicate the field (or column) separator.
For example, to indicate the data are in CSV format, you should use
`-d','`. The second useful flag is the `-f` flag, which indicates the
fields to extract from each row. With this flag, you can specify a
single column, a range of columns, or a set of columns. This is
demonstrated in the following code block, where we first extract only
column 17, next we extract columns 17 and 19, and finally we extract
columns 17 through 19, inclusively.

```console
$ cut -d',' -f17 2001.csv
$ cut -d',' -f17,19,21 2001.csv
$ cut -d',' -f17-19 2001.csv
```

You can try this command out in the following code cell.

-----

In [2]:
!head -5 /home/data_scientist/rppdm/data/2001.csv | cut -d',' -f17-19

Origin,Dest,Distance
BWI,CLT,361
BWI,CLT,361
BWI,CLT,361
BWI,CLT,361


-----

#### Character conversion

Another useful basic data processing task is character conversion, which
can be easily done via the `sed` command. `sed` is an abbreviation for
_stream editor_, which exactly describes what this command does, it
enables streaming data to be edited or converted on the fly. While this
command is extremely powerful, the basic substitution format is simple
(substitution is indicated by the `s` character. You specify a _pattern_
to match, followed by the _replacement_ text. By default only the first
occurrence on each line will be replaced, but you can specify to apply
the replacement globally, by appending a `g`. The different  components
are combined together as follows: `s/pattern/replacement/g` to indicate
that replacement should be substituted for pattern globally in the
stream. 

This command can be used in a Unix pipe or on a file, as demonstrated in
the following code block, where the first line flips only the first
occurrence of the pattern `ORD`, the second globally replaces all commas
with vertical bars, and the final one first replaces commas with spaces
before converting the character sequence `NA` to `null`.

```console
$ sed 's/ORD/DRO/' 2001.csv
$ sed 's/,/|/g' 2001.csv
$ sed 's/,/ /g' 2001.csv | sed 's/NA/null/g'
```

The following code blocks demonstrates this command on the airline data
set.

-----

In [3]:
%time !sed 's/NA/null/g' /home/data_scientist/rppdm/data/2001.csv | tail -5

2001,12,14,5,704,700,1159,1155,DL,678,-N914D,175,175,148,4,4,ONT,DFW,1189,14,13,0,null,0,null,null,null,null,null
2001,12,15,6,708,700,1158,1155,DL,678,-N913D,170,175,143,3,8,ONT,DFW,1189,9,18,0,null,0,null,null,null,null,null
2001,12,16,7,656,700,1147,1155,DL,678,-N910D,171,175,153,-8,-4,ONT,DFW,1189,7,11,0,null,0,null,null,null,null,null
2001,12,17,1,656,700,1151,1155,DL,678,N906D1,175,175,151,-4,-4,ONT,DFW,1189,13,11,0,null,0,null,null,null,null,null
2001,12,18,2,709,700,1158,1155,DL,678,N905D1,169,175,143,3,9,ONT,DFW,1189,10,16,0,null,0,null,null,null,null,null
CPU times: user 30 ms, sys: 0 ns, total: 30 ms
Wall time: 14 s


-----

This example reinforces the benefits of using Unix command line tools
for data processing. In under 15 seconds (on my system), we have
converted nearly six million rows of data!

----

### Advanced data processing

So far, the Unix commands we have used were fairly straightforward,
generally performing a single task. The next command, `awk`, is tool
like a swiss army knife with lots of capabilities. In fact, `awk` is a
basic programming language and can perform more complex operations
iteratively over every row in a file. The code for an `awk` program can
be specified on the command line, or alternatively be saved in a file
and executed. The basic approach `awk` employs is to parse a row of data
into fields (or columns), which are processed before moving on to the
next row. The fields in the current row can be accessed via their column
number, which simplifies data processing. In addition, you can specify
the delimiter that `awk` should use to parse a line into fields via the
`FS=""` flag.

As a programming language, `awk` supports conditional statements, which
can be used to branch based on the value of one or more fields,
functions, including built-ins like `print`, and variables and
operators. `awk` also has special `BEGIN` and `END` blocks to perform
operations before and after, respectively, parsing the rows in the file.
This can be useful for initializing variables or printing the output of
calculations that involved all rows. The following code block
demonstrate two `awk` examples. First, we use `awk` to print out columns
17 and 18 when the departure airport is equal to `ORD`. Second, we
compute the average distance of all flights that depart `CMI`.

```console
$ awk -v FS="," '{if($17 == "ORD") print "Flight Number: ", $10, $17" to "$18 ; }' 2001.csv
$ awk -v FS="," 'BEGIN{count = 0 ;}{if ($17 == "ORD") count += 1 ;} END{print count}' 2001.csv
```

We demonstrate another example in the following code cell, where we
compute the average distance for all flights that depart from Willard
airport. Since this statement is so long, we split the command over
multiple lines and use the backslash character to indicate that the line
continues on to the following line.

-----

In [4]:
%time !awk -v FS="," \
'BEGIN{sum >= $19 ; count = 0 ;} \
{if ($17 == "ORD") { sum += $19 ; count +=  1 } } \
END{print "Average distance = ", sum/count}' \
/home/data_scientist/rppdm/data/2001.csv

Average distance =  764.328
CPU times: user 20 ms, sys: 10 ms, total: 30 ms
Wall time: 10.2 s


-----

As this example demonstrates, `awk` can be extremely fast, taking less
than 11 seconds to compute the average distance of all flights departing
from `CMI`.

-----

## Breakout Session

During this breakout, you should gain experience working with Unix
commands presented in this lesson. Specific problems you can attempt
include the following:

1. Use 'head' top grab out the first 1000 lines from the 2001 Airline
data.

2. Pipe this data into the `wc` command to verify you have 1000 lines.

3. Pipe the frist command into `cut` to extract out the second, fourth,
and fifth columns.

Additional, more advanced problems:

1. Use `head` and `cut` to print out the first, third, and fourth
columns from the first 150 rows of the 2001 airline data. 

2. Use `sed`, `sort`, `awk`, and `uniq` in some order to build a
pipeline to print the number of airports located within each state. Note
the `-c` flag, which can be used with `uniq`, might be useful.

As time permits, return to the code cells in this notebook, make changes
and see the results (for example, change airport codes).

-----

### Additional References

1. The O'Reilly book, [Data Science at the Command Line][dscl]
2. [Unix Lessons][ul] from Practical Data Science Short Course

-----
[dscl]: http://datascienceatthecommandline.com
[ul]: https://github.com/ProfessorBrunner/rp-pdss15/blob/master/notebooks/1_unixdp.ipynb

### Return to the [Week One](index.ipynb) index.

-----